# Adjust index returns for inflation to get real returns

In [1]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
wb <- WorldBank()
indices <- Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date


Attaching package: ‘PerformanceAnalytics’

The following obje

### get rate of inflation from the WorldBank data-set

In [2]:
# see example: https://nbviewer.jupyter.org/github/shyams80/plutons/blob/master/docs-R/WorldBank.ipynb

wb$Meta() %>%
    filter(COUNTRY_NAME == "India" & INDICATOR_NAME %like% "CPI%") %>%
    select(INDICATOR_NAME, COUNTRY_KEY, INDICATOR_KEY) %>%
    print(n=Inf)

# Source:   lazy query [?? x 3]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockVizUs]
  INDICATOR_NAME                          COUNTRY_KEY INDICATOR_KEY
  <chr>                                         <int>         <int>
1 CPI Price, nominal                              135             3
2 CPI Price, nominal, seas. adj.                  135             5
3 CPI Price, % y-o-y, nominal, seas. adj.         135             6


In [3]:
# we need the CPI Price, % y-o-y, nominal, seas. adj.
wb$TimeSeries() %>%
    filter(COUNTRY_KEY == 135 & INDICATOR_KEY == 6) %>%
    arrange(YEAR) %>%
    print(n=Inf)

# Source:     lazy query [?? x 4]
# Database:   Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockVizUs]
# Ordered by: YEAR
   COUNTRY_KEY INDICATOR_KEY  YEAR VALUE
         <int>         <int> <int> <dbl>
 1         135             6  1987  8.79
 2         135             6  1988  9.40
 3         135             6  1989  3.27
 4         135             6  1990  8.97
 5         135             6  1991 13.9 
 6         135             6  1992 11.8 
 7         135             6  1993  6.36
 8         135             6  1994  6.34
 9         135             6  1995 11.3 
10         135             6  1996  8.90
11         135             6  1997  5.65
12         135             6  1998  4.47
13         135             6  1999  5.91
14         135             6  2000  5.99
15         135             6  2001  3.68
16         135             6  2002  5.20
17         135             6  2003  4.32
18         135             6  2004  3.85
19         135             6  2005  3.74
20         135   

### get NIFTY 50 total-return index returns from the Indices data-set

In [4]:
nifty50Dt <- indices$NseTimeSeries() %>%
    filter(NAME == 'NIFTY 50 TR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

annRets <- annualReturns(xts(nifty50Dt$CLOSE, nifty50Dt$TIME_STAMP))

print(annRets)

ERROR: Error in annualReturns(xts(nifty50Dt$CLOSE, nifty50Dt$TIME_STAMP)): could not find function "annualReturns"


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)